In [ ]:
REQUEST_ID = 188
AOI = 'POLYGON ((-91.93050383241031 34.50910284968832, -91.92844389588556 34.61032319084964, -92.074699389025 34.61145344841528, -92.07538603453924 34.51362929274102, -92.00328825622739 34.50966866851616, -92.00328825622739 34.50966866851616, -91.93050383241031 34.50910284968832))'
START_DATE = '2020-06-11'
END_DATE = '2020-07-13'

In [ ]:
import os
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio import features
from rasterio.merge import merge
from shapely.geometry import Polygon, shape, LinearRing
import shapely.wkt
from pathlib import Path
from datetime import datetime
import matplotlib.pylab as plt

from sklearn.cluster import KMeans
import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds, transform_resolution
from code.download.load_tiles import load_images
from code.index_research import combine_bands, calculate_ndvi, calculate_tcari, calculate_msr_g

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [ ]:
NB_USER = os.getenv('NB_USER')
BASE = f"/home/{NB_USER}/work"

API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/pbdnn")
PBD_DIR = os.path.join(BASE, "notebooks/pbdnn")

BANDS = {'B04','B08', 'TCI'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

for pbd_file in os.listdir(os.path.join(BASE, 'results/pbdnn')):
    if pbd_file.startswith(str(REQUEST_ID)+'_') or pbd_file.startswith('demo_'+str(REQUEST_ID)+'_'):
        PB_PATH = os.path.join(BASE, 'results/pbdnn', pbd_file)
        break
        
    else:
        # should be trigger PBDNN notebook excecution? PBDNN pipeline?
        PB_PATH = aoi_filename
        
PB_PATH

In [ ]:
date_tile_info = get_tiles(aoi_filename, sentinel_tiles_path)
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)

try:
    checked = get_min_clouds(checked)
except Exception:
    print(f'No clean raster found for period from {start_date} to {end_date}, skipping')
    
for i, tile in date_tile_info.iterrows():
    try:
        tile_folder = Path(checked[tile.tileID])
        print(f'filtered: {tile_folder}')
    except Exception as ex:
        print(ex)
        continue
    
    b04_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B04_10m.jp2' in filename][0]
    b08_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B08_10m.jp2' in filename][0]
    tci_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'TCI_10m.jp2' in filename][0]

In [ ]:
with rasterio.open(tci_tile) as src:
    tci_image = src.read()

In [ ]:
os.makedirs(f'{BASE}/results/dd/plant_stress/{REQUEST_ID}', exist_ok=True)
nir_green_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/nir_green.tif'
nir_green_path_upscaled = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/nir_green_upscaled.tif'
ndvi_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/ndvi.tif'

calculate_ndvi(b04_tile, b08_tile, out_path=ndvi_path, nodata=np.nan)

### NDVI vs nir_green clustering

In [ ]:
from collections import Counter

fields = gpd.read_file(PB_PATH)[:100]
thresh = 0.5
all_polys = gpd.GeoDataFrame()
num_clusters = 2
tci_images, pred_ndvi, ndvi_imgs, clustered = [], [], [], []

for idx, row in tqdm(fields.iterrows(), total=len(fields)):
    
    with rasterio.open(tci_tile) as src:
        tci_image, tfs = riomask.mask(
            src, [fields.to_crs(src.crs).iloc[idx].geometry], all_touched=False, crop=True)

    with rasterio.open(ndvi_path) as src:
        ndvi, tfs = riomask.mask(
            src, [fields.to_crs(src.crs).iloc[idx].geometry], all_touched=False, crop=True)
        crs = src.crs
        
    raster_mask = ndvi.astype(np.bool)
    ndvi_img = np.where(raster_mask, ndvi, 0)
    scanner_ndvi = KMeans(n_clusters=num_clusters)
        
    try:
        scanner_ndvi.fit(ndvi_img[raster_mask].reshape(-1, 1))
    except Exception as e:
        print(e, 'shape:', ndvi.reshape(-1, 1).shape)
        continue
        
    c, h, w = ndvi.shape
    labels = scanner_ndvi.labels_ #reshape(h, w)
    ndvi_img = ndvi_img.ravel()
    ndvi_img[raster_mask.ravel()] = labels + 1
    ndvi_img = ndvi_img.reshape(h, w)

    #anomaly_vals = np.min(ndvi_img[np.nonzero(ndvi_img)])
    img = ndvi_img
    
    cluster_mask1 = np.where(img==1, ndvi, 0)
    cluster_mask2 = np.where(img==2, ndvi, 0)
    
    cluster_val1 = cluster_mask1.mean(where=cluster_mask1!=0)
    cluster_val2 = cluster_mask2.mean(where=cluster_mask2!=0)
    
    
    anomaly_val = 1 if cluster_val1 < cluster_val2 else 2
    cl_img = np.where(img==anomaly_val, 1, 0)
    
    if img.sum() == 0:
        print(f'No anomalies found for field {idx}')
        continue
    
    try:
        polys = list(features.shapes(img, transform=tfs))
    except Exception as e:
        print(e, 'image shape:', img.shape)
        continue

    tci_images.append(tci_image)
    ndvi_imgs.append(ndvi)
    pred_ndvi.append(img)
    clustered.append(cl_img)

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def visualize_results(tci_images, ndvi_images, pred_images, clustered_images):
    def visualize_image(num):
        tci = tci_images[num]
        ndvi = ndvi_images[num]
        pred = pred_images[num]
        clustered = clustered_images[num]

        return tci, ndvi, pred, clustered

    def f_fabric(num):
        def f(x):
            tci, ndvi, pred, clustered = visualize_image(x)

            fig, axes = plt.subplots(2, 2, figsize = (15, 12))

            axes[0][0].set_title('TCI')
            axes[0][0].imshow(tci.transpose(1, 2, 0))

            axes[0][1].set_title('NDVI')
            axes[0][1].imshow(ndvi[0], cmap='gray')
            
            axes[1][0].set_title('pred ndvi')
            axes[1][0].imshow(pred, cmap='gray')
            
            axes[1][1].set_title('clustered')
            axes[1][1].imshow(clustered, cmap='gray')

            plt.tight_layout()

        return f
    
    plt.rcParams.update({'font.size': 14})
    f = f_fabric(0)
    interact(f, x=widgets.IntSlider(min=0, max=len(tci_images)-1, step=1, value=0, continuous_update=False))
    
visualize_results(tci_images, ndvi_imgs, pred_ndvi, clustered)

In [ ]:
remove_aux_rasters = False

if remove_aux_rasters:
    
    try:
        os.remove(nir_green_path)
    except FileNotFoundError:
        print('No helping rasters found')
        
    try:
        os.remove(nir_green_path_upscaled)
    except FileNotFoundError:
        print('No helping rasters found')
    
    try:
        os.rmdir(tile_folder)
    except FileNotFoundError:
        print(f'Error removing directory {tile_folder}')